
# TP Image classification of MNIST (reduced)
## using correlation filters, kNN and/or logistic regression
### Diane LINGRAND 

diane.lingrand@univ-cotedazur.fr   
Polytech SI3  2022-2023

In [ ]:
from IPython.display import Image
import matplotlib.pyplot as plt
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay

from tqdm import tqdm

import pickle

## Data loading

Before executing the next cell, you need to have downloaded the files in the current directory or to change the paths in the next cell.

In [ ]:
import pickle
with open('redMNIST-x-train.bin', 'rb') as input:
    xTrain = pickle.load(input)
with open('redMNIST-x-test.bin', 'rb') as input:
    xTest = pickle.load(input)
with open('redMNIST-y-test.bin', 'rb') as input:
    yTest = pickle.load(input)
with open('redMNIST-y-train.bin', 'rb') as input:
    yTrain = pickle.load(input)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

**Question 1:** What are the dimensions of train data and test data?

In [ ]:
# your work

**Question 2:** How many classes?

In [ ]:
# your work

**Question 3:** What are the minimal and maximal values of pixels in those images?

In [ ]:
# your work

Let's visualize an image from the training dataset:

In [ ]:
import random
n = random.randrange(0,len(yTrain))
plt.imshow(xTrain[n], cmap=plt.cm.gray)

## Baseline: raw MNIST data without correlations

We first need to flatten the data:

In [ ]:
x_train = xTrain.reshape(len(yTrain),784)
x_test = xTest.reshape(len(yTest),784)

The baseline is obtained using a kNN classification with k=10 on the raw MNIST dataset. 

**Question 4:** Compute the evaluation metrics (accuracy, F1 score and confusion matrix) using the test set. Publish the accuracy in slack as the variable <font color='red'>rawKnn10</font>.

In [ ]:
# your work

**Question 5:** Find the best k value for the previous question. Publish the best k value as <font color='red'>rawKnnBestK</font> and the best accuracy as <font color='red'>rawKnnBestAcc</font>.

In [ ]:
# your work

**Question 5:** Learn a logistic regression on the train set and compute the metrics on the test set. Be aware that the default maximal number of iterations may need to be adapted. Publish the accuracy as <font color='red'>rawLogReg</font>.

In [ ]:
# your work

**Question 6:** Up to now, what is the best method?

In [ ]:
# your work

## Correlation kernels and functions

We propose 2 sets of kernel denoted by variables 'c3x3' and 'c5x5'. Feel free to imagine other sets of kernels.

In [ ]:
# 4 kernels of dim 3x4
c0 = np.array([[-1.1, -1.49, -0.72],
 [-0.78,  0.8,   0.92],
 [ 1.58,  0.73,  0.62]])
c1 = np.array([[ 1.22, 0.97,  0.39],
 [-0.62,  0.92,  1.42],
 [-1.55, -1.65, -0.51]])
c2 = np.array([[ 0.48, 1.32, -1.08],
 [ 0.98,  0.69, -1.99],
 [ 1.35, -0.24, -1.59]])
c3 = np.array([[-0.38,  1.03,  1.28],
 [-1.37,  0.18,  0.24],
 [-0.1,   0.9,   1.29]])
c3x3 = np.array([c0,c1,c2,c3])

In [ ]:
# 8 kernels of dim 5x5
c0 = np.array([[-0.05, -0.13,  0.02,  0.12,  0.12],
 [-0.23,  0.45,  0.86,  0.79,  0.35],
 [-0.04,  0.35,  0.58,  0.41, -0.4 ],
 [-0.09,  0.12,  0.2 ,  0.22, -0.6 ],
 [ 0.31,  0.07, -0.01, -0.38, -0.19]])
c1 = np.array([[ 0.27,  0.1 ,  0.36,  0.29,  0.65],
 [ 0.34,  0.54,  0.62,  0.43,  0.76],
 [ 0.66,  0.64,  0.57,  0.15, -0.06],
 [ 0.25,  0.29, -0.01, -0.14, -0.84],
 [-1.23, -1.59, -1.78, -1.04, -1.07]])
c2 = np.array([[-0.66, -0.87, -1.09, -1.25, -0.44],
 [-0.01,  0.18, -0.06,  0.11,  0.13],
 [ 0.39,  0.5 ,  0.51,  0.42,  0.32],
 [ 0.32,  0.49,  0.3 ,  0.35,  0.41],
 [ 0.64,  0.7 ,  0.22, -0.05,  0.28]])
c3 = np.array([[ 1.01,  0.07,  0.16, -0.42, -0.75],
 [ 0.23,  0.08,  0.3 ,  0.11, -0.14],
 [-0.69,  0.02,  0.19,  0.35,  0.16],
 [-0.74,  0.45,  0.61,  0.26, -0.04],
 [ 0.1,   0.6 ,  0.15, -0.37, -0.84]])
c4 = np.array([[ 0.25,  0.34,  0.29, -0.42, -0.69],
 [ 0.5,   0.31,  0.45, -0.34, -0.31],
 [ 0.66,  0.6 ,  0.28, -0.62, -0.82],
 [ 0.37,  0.73, -0.05, -0.58, -0.87],
 [ 0.59,  0.43,  0.02, -0.62, -0.41]])
c5 = np.array([[ 0.06, -0.48, -0.43, -0.93, -0.19],
 [ 0.57,  0.63,  0.5 ,  0.02, -0.35],
 [ 0.5 ,  0.92,  1.03,  0.33, -0.58],
 [-0.55,  0.11,  0.77,  0.74,  0.21],
 [-1.44, -1.08, -0.54,  0.01,  0.27]])
c6 = np.array([[ 0.4 ,  0.62,  0.73,  0.58,  0.43],
 [-0.02,  0.47,  0.99,  0.55,  0.65],
 [-1.01, -0.83,  0.07,  0.51,  0.47],
 [-1.01, -1.16, -0.8 , -0.06,  0.5 ],
 [-0.06, -0.24, -0.96, -0.7 , -0.08]])
c7 = np.array([[-0.8 , -0.25, -0.02,  0.36,  0.4 ],
 [-1.43, -0.01,  0.04,  0.28,  0.63],
 [-1.3 ,  0.1 ,  0.61,  0.58,  0.4 ],
 [-1.28,  0.23,  0.46,  0.34,  0.35],
 [-0.57,  0.44,  0.51,  0.34, -0.  ]])
c5x5 = np.array([c0,c1,c2,c3,c4,c5,c6,c7])

In [ ]:
#filters display
def displayFilters(filtres):
    nbFiltres, dimFiltres = filtres.shape[:2]

    fig, axes = plt.subplots(nrows=1, ncols=nbFiltres,figsize=(nbFiltres*4,6))
    ax = axes.ravel()
    for k in range(nbFiltres):
        ax[k].matshow(filtres[k], cmap = plt.cm.gray)
        poids = filtres[k]
        poids = np.round(poids*100)/100
   
        for i in range(dimFiltres):
            for j in range(dimFiltres):
                if filtres[k,i,j] < 0:
                    text = ax[k].text(j, i, poids[i,j], ha="center", va="center", color="w")
                else:
                    text = ax[k].text(j, i, poids[i,j], ha="center", va="center", color="k")


In [ ]:
displayFilters(c3x3)

In [ ]:
displayFilters(c5x5)

**Question 7:** Copy/paste here your best version of a function for correlation without normalisation.

In [ ]:
# your work

**Question 8:** Observe the results of correlation using a set of filters on a single image from the dataset.

In [ ]:
# your work

**Question 9:** Similar question but adding a new parameter: the stride. Test also on a single image as you did in the previous question.

In [ ]:
# your work

**Question 10:** Write a fonction that will compute all the correlations on a dataset x using a list of filters c with a stride similar on vertical and horizontal. We assume that all filters are squared filters with odd dimensions. The result is a transformed dataset with the same number of data. Each data is a vector containing all results of correlations.

In [ ]:
def passeLesFiltres(x,c,stride):
   
    # your work

    return newX

**Question 11:** Apply the previous function to the train and test dataset using the set of filters c3x3 and a stride of 3. Print the shape of the results.

In [ ]:
fftrain3 = # tbd
fftest3 = # tbd

**You know will use such transformed dataset in order to improve the classification results.**

## Improved classifications

Using the metrics computed on the test set, try different combinations:
- kNN (which k) or Logistic Regression
- correlation filters: c3x3, c5x5 or other filters from your imagination
- normalisation or not for the correlation
- values of strides
- removing negative values after the correlation operation
Or any other idea that you can discuss with your teachers (or in the slack channel of the course). 

Each time you obtain a better accuracy than in the previous questions, publish it in slack.

In [ ]:
# your work